# Assignment 2. Language modeling.

This task is devoted to language modeling. Its goal is to write in PyTorch an RNN-based language model. Since word-based language modeling requires long training and is memory-consuming due to large vocabulary, we start with character-based language modeling. We are going to train the model to generate words as sequence of characters. During training we teach it to predict characters of the words in the training set.



## Task 1. Character-based language modeling: data preparation (15 points).

We train the language models on the materials of **Sigmorphon 2018 Shared Task**. First, download the Russian datasets.

In [1]:
!wget https://raw.githubusercontent.com/sigmorphon/conll2018/master/task1/surprise/russian-train-high
!wget https://raw.githubusercontent.com/sigmorphon/conll2018/master/task1/surprise/russian-dev
!wget https://raw.githubusercontent.com/sigmorphon/conll2018/master/task1/surprise/russian-test

--2020-04-01 19:24:17--  https://raw.githubusercontent.com/sigmorphon/conll2018/master/task1/surprise/russian-train-high
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.112.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.112.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 533309 (521K) [text/plain]
Saving to: ‘russian-train-high’

russian-train-high  100%[===================>] 520.81K  1.86MB/s    in 0.3s    

2020-04-01 19:24:17 (1.86 MB/s) - ‘russian-train-high’ saved [533309/533309]

--2020-04-01 19:24:17--  https://raw.githubusercontent.com/sigmorphon/conll2018/master/task1/surprise/russian-dev
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.112.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.112.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53671 (52K) [text/plain]
Saving to: ‘russian-dev’

russian-dev  

**1.1 (1 points)**
All the files contain tab-separated triples ```<lemma>-<form>-<tags>```, where ```<form>``` may contain spaces (*будете соответствовать*). Write a function that loads a list of all word forms, that do not contain spaces.  

In [2]:
import csv
def read_infile(infile):
    lower = False
    words = list()
    with open(infile, newline = '') as games:                                                                                          
        line = csv.reader(games, delimiter='\t')
        for word in line:
            if ' ' not in word[1]:
                if lower:
                    words.append(word[1].lower())
                else:
                    words.append(word[1])
    return words

In [3]:
train_words = read_infile("russian-train-high")
dev_words = read_infile("russian-dev")
test_words = read_infile("russian-test")
print(len(train_words), len(dev_words), len(test_words))
print(*train_words[:10])

9213 917 922
валлонскому незаконченным истрёпывав личного серьгам необоснованным тюти заросла идеальна гулкой


**1.2 (2 points)** Write a **Vocabulary** class that allows to transform symbols into their indexes. The class should have the method ```__call__``` that applies this transformation to sequences of symbols and batches of sequences as well. You can also use [SimpleVocabulary](https://github.com/deepmipt/DeepPavlov/blob/c10b079b972493220c82a643d47d718d5358c7f4/deeppavlov/core/data/simple_vocab.py#L31) from DeepPavlov. Fit an instance of this class on the training data.

In [4]:
from deeppavlov.core.data.simple_vocab import SimpleVocabulary
"""
== YOUR CODE HERE ==
"""        
vocab = SimpleVocabulary(save_path='.', special_tokens=("END", "BEGIN", "UNK", "PAD"))
vocab.fit([list(x) for x in train_words])
print(len(vocab))

2020-04-01 19:24:21.470 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 49: No load path is set for SimpleVocabulary in 'infer' mode. Using save path instead


53


In [5]:
vocab(list(train_words[1])), list(train_words[1])

([8, 6, 23, 5, 15, 4, 8, 26, 6, 8, 8, 18, 14],
 ['н', 'е', 'з', 'а', 'к', 'о', 'н', 'ч', 'е', 'н', 'н', 'ы', 'м'])

**1.3 (2 points)** Write a **Dataset** class, which should be inherited from ```torch.utils.data.Dataset```. It should take a list of words and the ```vocab``` as initialization arguments.

In [6]:
import torch
from torch.utils.data import Dataset as TorchDataset

class Dataset(TorchDataset):
    """Custom data.Dataset compatible with data.DataLoader."""
    def __init__(self, data, vocab):
        self.data = data
        self.vocab = vocab

    def __getitem__(self, idx):
        """
        Returns one tensor pair (source and target). The source tensor corresponds to the input word,
        with "BEGIN" and "END" symbols attached. The target tensor should contain the answers
        for the language model that obtain these word as input.        
        """
        source_target = torch.LongTensor(self.vocab(["BEGIN"] + list(self.data[idx]) + ["END"]))
        source = source_target[:-1]
        target = source_target[1:]
        return (source, target)
        
    def __len__(self):
        return len(self.data)

In [7]:
train_dataset = Dataset(train_words, vocab)
dev_dataset = Dataset(dev_words, vocab)
test_dataset = Dataset(test_words, vocab)

**1.4 (3 points)** Use a standard ```torch.utils.data.DataLoader``` to obtain an iterable over batches. Print the shape of first 10 input batches with ```batch_size=1```.

In [8]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=1)
dev_loader = DataLoader(dev_dataset, batch_size=1)
test_loader = DataLoader(test_dataset, batch_size=1)

In [9]:
batcher = iter(train_loader)
for i in range(10):
    (x, y) = next(batcher)
    print(i, x.shape, x.dtype, y.shape, y.dtype)

0 torch.Size([1, 12]) torch.int64 torch.Size([1, 12]) torch.int64
1 torch.Size([1, 14]) torch.int64 torch.Size([1, 14]) torch.int64
2 torch.Size([1, 11]) torch.int64 torch.Size([1, 11]) torch.int64
3 torch.Size([1, 8]) torch.int64 torch.Size([1, 8]) torch.int64
4 torch.Size([1, 8]) torch.int64 torch.Size([1, 8]) torch.int64
5 torch.Size([1, 15]) torch.int64 torch.Size([1, 15]) torch.int64
6 torch.Size([1, 5]) torch.int64 torch.Size([1, 5]) torch.int64
7 torch.Size([1, 8]) torch.int64 torch.Size([1, 8]) torch.int64
8 torch.Size([1, 9]) torch.int64 torch.Size([1, 9]) torch.int64
9 torch.Size([1, 7]) torch.int64 torch.Size([1, 7]) torch.int64


**(1.5) 1 point** Explain, why this does not work with larger batch size.

**(1.6) 5 points** Write a function **collate** that allows you to deal with batches of greater size. See [discussion](https://discuss.pytorch.org/t/dataloader-for-various-length-of-data/6418/8) for an example. Implement your function as a class ```__call__``` method to make it more flexible.

In [10]:
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_sequence

def pad_tensor(vec, length, dim, pad_symbol):
    """
    Pads a vector ``vec`` up to length ``length`` along axis ``dim`` with pad symbol ``pad_symbol``.
    """
    pad_size = list(vec.shape)
    pad_size[dim] = length - vec.size(dim)
    return torch.cat((vec.type(torch.int64), torch.ones(*pad_size, dtype=torch.int64) * pad_symbol), dim=dim)

class Padder:
    def __init__(self, dim=0, pad_symbol=0): 
        self.dim = dim
        self.pad_symbol = pad_symbol
        
    def __call__(self, batch):
        max_length_x = max([len(x) for x,_ in batch]) 
        max_length_y = max([len(y) for _,y in batch])
        x = torch.stack([pad_tensor(x, max_length_x, dim=self.dim, pad_symbol=self.pad_symbol) for x,_ in batch], 0) 
        y = torch.stack([pad_tensor(y, max_length_y, dim=self.dim, pad_symbol=self.pad_symbol) for _,y in batch], 0) 
        return x, y

**(1.7) 1 points** Again, use ```torch.utils.data.DataLoader``` to obtain an iterable over batches. Print the shape of first 10 input batches with the batch size you like.

In [11]:
from torch.utils.data import DataLoader
padder = Padder(dim=0, pad_symbol=vocab['PAD'])
train_loader = DataLoader(train_dataset, batch_size=8, collate_fn=Padder(dim=0))

In [12]:
batcher = iter(train_loader)
for i in range(10):
    batch = next(batcher)
    print(i, batch[0].shape, batch[0].dtype, batch[1].shape, batch[1].dtype)

0 torch.Size([8, 15]) torch.int64 torch.Size([8, 15]) torch.int64
1 torch.Size([8, 15]) torch.int64 torch.Size([8, 15]) torch.int64
2 torch.Size([8, 14]) torch.int64 torch.Size([8, 14]) torch.int64
3 torch.Size([8, 16]) torch.int64 torch.Size([8, 16]) torch.int64
4 torch.Size([8, 18]) torch.int64 torch.Size([8, 18]) torch.int64
5 torch.Size([8, 16]) torch.int64 torch.Size([8, 16]) torch.int64
6 torch.Size([8, 14]) torch.int64 torch.Size([8, 14]) torch.int64
7 torch.Size([8, 11]) torch.int64 torch.Size([8, 11]) torch.int64
8 torch.Size([8, 14]) torch.int64 torch.Size([8, 14]) torch.int64
9 torch.Size([8, 11]) torch.int64 torch.Size([8, 11]) torch.int64


## Task 2. Character-based language modeling. (35 points)

**2.1 (5 points)** Write a network that performs language modeling. It should include three layers:
1. **Embedding** layer that transforms input symbols into vectors.
2. An **RNN** layer that outputs a sequence of hidden states (you may use https://pytorch.org/docs/stable/nn.html#gru).
3. A **Linear** layer with ``softmax`` activation that produces the output distribution for each symbol.

In [13]:
import torch.nn as nn

class RNNLM(nn.Module):
    def __init__(self, vocab_size, embeddings_dim, hidden_size):
        super(RNNLM, self).__init__()
        self.embeddings = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embeddings_dim)
        self.rnn = nn.GRU(embeddings_dim, hidden_size, batch_first=True)
        self.linear = nn.Linear(hidden_size, vocab_size)
        self.softmax = nn.Softmax(dim=2)
        
    def forward(self, inputs, hidden=None):
        x = self.embeddings(inputs)
        y, hidden = self.rnn(x, hidden)
        logits = self.linear(y)
        prob = self.softmax(logits)
        return logits, prob

**2.2 (1 points)** Write a function ``validate_on_batch`` that takes as input a model, a batch of inputs and a batch of outputs, and the loss criterion, and outputs the loss tensor for the whole batch. This loss should not be normalized.

In [14]:
def validate_on_batch(model, criterion, x, y):
    linear, prob = model(x)
    loss = 0
    for i in range(linear.shape[1]):
        padding_mask = torch.tensor(y[:, i] != padder.pad_symbol, dtype=int)
        loss += (criterion(linear[:, i, :], y[:, i]) * padding_mask).sum() / padding_mask.sum()
    loss = loss / linear.shape[1]
    return loss

**2.3 (1 points)** Write a function ``train_on_batch`` that accepts all the arguments of ``validate_on_batch`` and also an optimizer, calculates loss and makes a single step of gradient optimization. This function should call ``validate_on_batch`` inside.

In [15]:
def train_on_batch(model, criterion, x, y, optimizer):
    optimizer.zero_grad()
    loss = validate_on_batch(model, criterion, x, y)
    loss.backward()
    optimizer.step()
    return loss

**2.4 (3 points)** Write a training loop. You should define your ``RNNLM`` model, the criterion, the optimizer and the hyperparameters (number of epochs and batch size). Then train the model for a required number of epochs. On each epoch evaluate the average training loss and the average loss on the validation set. 

**2.5 (3 points)** Do not forget to average your loss over only non-padding symbols, otherwise it will be too optimistic.

In [16]:
EPOCHS = 10
BATCH_SIZE = 24
VOCAB_SIZE = len(vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 200
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-2

In [17]:
padder = Padder(dim=0, pad_symbol=vocab['PAD'])
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=Padder(dim=0))
dev_dataloader = DataLoader(dev_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=Padder(dim=0))

model = RNNLM(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_DIM)

criterion = nn.CrossEntropyLoss(reduction='none')
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

In [18]:
losses_train = list()
losses_val = list()
for epoch in range(EPOCHS):
    
    train_loss = 0
    model.train()
    for i_batch, batch in enumerate(train_dataloader):
        loss = train_on_batch(model, criterion, batch[0], batch[1], optimizer)
        train_loss += loss.item()
    train_loss = train_loss / len(train_dataloader)
    losses_train.append(train_loss)
    
    val_loss = 0
    model.eval()
    for i_batch, batch in enumerate(dev_dataloader):
        with torch.no_grad():
            loss = validate_on_batch(model, criterion, batch[0], batch[1])
        val_loss += loss.item()
    val_loss = val_loss / len(dev_dataloader)
    losses_val.append(val_loss)
    
    print('Epoch: {}, Training Loss: {}, Validation Loss: {}'.format(epoch, train_loss, val_loss))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """


Epoch: 0, Training Loss: 1.587787357935061, Validation Loss: 1.4095344512890546
Epoch: 1, Training Loss: 1.357682776171714, Validation Loss: 1.3466459512710571
Epoch: 2, Training Loss: 1.2991921721647184, Validation Loss: 1.3125305328613672
Epoch: 3, Training Loss: 1.2572489061082404, Validation Loss: 1.2894700200129778
Epoch: 4, Training Loss: 1.221893868719538, Validation Loss: 1.2735766172409058
Epoch: 5, Training Loss: 1.190219511743635, Validation Loss: 1.2626772599342542
Epoch: 6, Training Loss: 1.1609363177170355, Validation Loss: 1.2550962918843978
Epoch: 7, Training Loss: 1.1333522057781618, Validation Loss: 1.250234270707155
Epoch: 8, Training Loss: 1.107228999491781, Validation Loss: 1.2478399230883672
Epoch: 9, Training Loss: 1.0825903185953696, Validation Loss: 1.2483548858226874


In [19]:
import matplotlib.pyplot as plt

plt.plot(losses_train)
plt.plot(losses_val)
plt.show();

<Figure size 640x480 with 1 Axes>

**2.6 (5 points)** Write a function **predict_on_batch** that outputs letter probabilities of all words in the batch.

In [20]:
def predict_on_batch(model, batch):
    with torch.no_grad():
        linear, prob = model(batch[0])
    probs = torch.ones(batch[1].shape)
    for i in range(batch[1].shape[0]): 
        for j in range(batch[1].shape[1]):
            probs[i, j] = prob[i, j, batch[1][i, j]]
    return probs

**2.7 (1 points)** Calculate the letter probabilities for all words in the test dataset. Print them for 20 last words. Do not forget to disable shuffling in the ``DataLoader``.

In [22]:
test_prob = []
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=Padder(dim=0))

model.eval()
for batch in test_dataloader:
    prob = predict_on_batch(model, batch)
    prob = prob.numpy().tolist()
    test_prob += prob

for i in range(-20, 0):
    word = test_words[i]
    print('Word: ', word, '; Letters probability:')
    for j, letter in enumerate(word):
        print("%s %.3f" % (letter, test_prob[i][j]), end='\n')
    print('======'*10)

Word:  пшеничной ; Letters probability:
п 0.110
ш 0.000
е 0.454
н 0.026
и 0.054
ч 0.044
н 0.364
о 0.281
й 0.467
Word:  переживаниями ; Letters probability:
п 0.110
е 0.089
р 0.859
е 0.856
ж 0.015
и 0.376
в 0.277
а 0.944
н 0.294
и 0.651
я 0.634
м 0.237
и 0.268
Word:  наибольшим ; Letters probability:
н 0.066
а 0.447
и 0.023
б 0.018
о 0.052
л 0.089
ь 0.458
ш 0.005
и 0.453
м 0.111
Word:  правителями ; Letters probability:
п 0.110
р 0.408
а 0.038
в 0.544
и 0.144
т 0.036
е 0.365
л 0.717
я 0.021
м 0.459
и 0.191
Word:  апеллируешь ; Letters probability:
а 0.048
п 0.029
е 0.046
л 0.032
л 0.005
и 0.229
р 0.104
у 0.429
е 0.138
ш 0.153
ь 0.974
Word:  односложных ; Letters probability:
о 0.084
д 0.041
н 0.341
о 0.892
с 0.153
л 0.168
о 0.088
ж 0.273
н 0.464
ы 0.398
х 0.231
Word:  колодками ; Letters probability:
к 0.051
о 0.466
л 0.092
о 0.522
д 0.088
к 0.019
а 0.182
м 0.336
и 0.511
Word:  вибрирует ; Letters probability:
в 0.080
и 0.038
б 0.004
р 0.213
и 0.522
р 0.011
у 0.338
е 0.269
т 0.300
Word:

**2.8 (5 points)** Write a function that generates a single word (sequence of indexes) given the model. Do not forget about the hidden state! Be careful about start and end symbol indexes. Use ``torch.multinomial`` for sampling.

In [23]:
def generate(model, max_length=20, start_index=1, end_index=2):
    model.eval()
    x = torch.LongTensor([vocab(['BEGIN'])])
    for i in range(max_length):
        with torch.no_grad():
            linear, prob = model(x)
        next_ = torch.multinomial(prob[0][-1], num_samples=1)
        if next_.item() in vocab(['BEGIN', 'END', 'PAD', 'UNK']):
            break
        x = torch.cat([x[0], next_]).unsqueeze(0)
    return x[0][1:]

**2.9 (1 points)** Use ``generate`` to sample 20 pseudowords. Do not forget to transform indexes to letters.

In [24]:
for i in range(20):
    word = ''.join([vocab[i] for i in generate(model).numpy().tolist()])
    print(word)

илевлекты
шут
обельца
короворционат
допотеньким
оторествам
термину
лесохианские
выетдавший
растрёрному
трудозаистым
поспет
заёбрызный
сгелями
слогоционфасным
сопымив
осугильют
струдительного
изриста
бубликам


**(2.10) 5 points** Write a batched version of the generation function. You should sample the following symbol only for the words that are not finished yet, so apply a boolean mask to trace active words.

**(2.11) 5 points** Experiment with the type of RNN, number of layers, units and/or dropout to improve the perplexity of the model.